<a href="https://colab.research.google.com/github/PieroPastor/quantum-chess-bot-player/blob/main/Quantum_Chess_Monte_Carlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Monte Carlo to plays Quantum Chess

Este notebook se encargará de crear un algoritmo de montecarlo limitado, para ahorrar recursos, que pueda jugar de manera mínimamente aceptable al ajedrez cuántico. Luego se hará jugar a dos de estos algoritmos, uno contra uno, para conseguir de este juego un dataset para la red neuronal del otro notebook. La forma en la que se utilizará el montecarlo es para tener un input, output para el aprendizaje de la red neuronal.

Input: Arreglo del tablero con la versión ASCII del símbolo de la pieza, en positivo o negativo según su color. Y al principio el color de a quien le toca jugar [0 = Negras, 1 = Blancas]

Output: Movimiento (Arreglo numérico o Vector).
        
    output = [Valor1, Valor2, Valor3, Valor4, Valor5, Valor6]
    Valor 1 = Movimiento a realizar (1 : Move, 2 : Split, 3 : Merge)
    Valor 2 = Posición inicio (tablero[7][2] -> 7*2+2 = 16)
    Valor 3 = Posición inicio (tablero[7][2] -> 7*2+2 = 16, Si no es Merge es -1)
    Valor 4 = Posición objetivo (tablero[7][2] -> 7*2+2 = 16)
    Valor 5 = Posición objetivo (tablero[7][2] -> 7*2+2 = 16, Si no es Split es -1)
    Valor 6 = A que coronar (0 : Nada, 1 : Torre, 2 : Dama, 3 : Alfil, 4 : Caballo)

    Por ejemplo: [1, 15, -1, 23, -1, 0] : Hace Move de 15 a 23, como no es Split no necesita una posición extra de llegada, como no hay coronación es 0.


## Carga del Quantum Chess desde GitHub

In [ ]:
! git clone https://github.com/PieroPastor/quantum-chess-bot-player.git

fatal: destination path 'quantum-chess-bot-player' already exists and is not an empty directory.


##Carga de librerias Necesarias

In [ ]:
! pip install -r quantum-chess-bot-player/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.6/152.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.7/532.7 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Importación de los .py del proyecto del juego.

In [ ]:
import sys
sys.path.append('/content/quantum-chess-bot-player/QuantumChess')  # Ajusta la ruta si es necesario
import Tablero
import Piezas
from Piezas import BColors
import csv
##################################
from collections import namedtuple
from Tablero import Tablero
import random
import itertools
import copy

pygame 2.6.0 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Clase Juego

Clase genérica donde estarán los métodos genéricos que controlaran la base del juego.

In [ ]:
class Game:
  def actions(self, state):
    """Retorna una lista de movidas permitidas en el estado actual state."""
    raise NotImplementedError

  def result(self, state, move):
    """Retorna el nuevo estado que resulta de hacer una movida move en el estado state."""
    raise NotImplementedError

  def utility(self, state, player):
    """Retorna el valor de utilidad para el jugador player en el estado terminal state."""
    raise NotImplementedError

  def terminal_test(self, state):
    """Retorna True si el estado state es un estado terminal del juego."""
    return not self.actions(state)

  def to_move(self, state):
    """Retorna el jugador que le toca jugar en el presente estado state."""
    return state.to_move

  def display(self, state):
    """Imprime o displaya el state."""
    print(state)

  def __repr__(self):
    return '<{}>'.format(self.__class__.__name__)

  def play_game(self, *players, verbose):
    """Controlador del juego:
    Llama alternadamente a cada jugador pasandole el estado actual del juego y ejecutando la movida retornada."""
    state = self.initial
    numJugada = 0
    while True:
      for player in players:
        move = player(self, state)
        mark_now = self.to_move(state)
        state = self.result(state, move)
        numJugada = numJugada + 1
        if verbose:
          print("Jugada", numJugada, ": Turno del jugador", player.__name__, "(",mark_now,")")
          self.display(state)
          print("*************************************************")
        if self.terminal_test(state):
          print("Jugada", numJugada, "(final): Turno del jugador", player.__name__, "(",mark_now,")")
          self.display(state)
          print("La utilidad del primer jugador (",self.to_move(self.initial),") fue: ")
          #retorna utilidad del 1er jugador al acabar el juego
          return self.utility(state, self.to_move(self.initial))

  def convert_pieces(self, board):
    tablero = board.tablero
    probabilidades = board.probabilidades
    t = []
    for row in tablero:
      for i in row:
        if i == ".": t.append(0)
        else:
          p = i[5]
          if i[0:5] == BColors.BLACK: mult = -1
          else: mult = 1
          t.append(ord(p)*mult)
    return t
    for row in probabilidades:
      for i in row:
        t.append(i)

  def convert_move(self, move):
    neo = [move[0]]
    for i in range(1, 5):
      if not isinstance(move[i], int): neo.append(move[i][0]*8+move[i][1])
      else: neo.append(-1)
    neo.append(move[5])
    return neo

  def play_save_game(self, *players, verbose, delete_random=False):
    """Controlador del juego:
    Llama alternadamente a cada jugador pasandole el estado actual del juego y ejecutando la movida retornada."""
    state = self.initial
    numJugada = 0
    nombre_archivo = "dataset.csv"
    with open(nombre_archivo, mode="w", newline="") as archivo:
      escritor_csv = csv.writer(archivo)
      while True:
        for player in players:
          move = player(self, state)
          input_board = [1 if state.to_move == "W" else 0]
          input_board += self.convert_pieces(state.board)
          mark_now = self.to_move(state)
          state = self.result(state, move)
          output_move = self.convert_move(move)
          if not delete_random: escritor_csv.writerow(input_board+output_move)
          if delete_random and not player.__name__ == "random_player":
            escritor_csv.writerow(input_board+output_move)
          numJugada = numJugada + 1
          if verbose:
            print("Jugada", numJugada, ": Turno del jugador", player.__name__, "(",mark_now,")")
            self.display(state)
            print("*************************************************")
          if self.terminal_test(state):
            print("Jugada", numJugada, "(final): Turno del jugador", player.__name__, "(",mark_now,")")
            self.display(state)
            print("La utilidad del primer jugador (",self.to_move(self.initial),") fue: ")
            #retorna utilidad del 1er jugador al acabar el juego
            return self.utility(state, self.to_move(self.initial))

##QuantumChess Class

In [ ]:
#Para este entorno, un estado es una tupla con nombres de campos (namedtuple)
GameState = namedtuple('GameState', 'to_move, utility, board, moves')

class QuantumChess(Game):
  def __init__(self):
    self.tab = Tablero()
    moves = self.tab.GetMoves('W')
    self.initial = GameState(to_move='W', utility=0, board=self.tab, moves=moves)

  #Cargará los movimientos posibles para el que le toque, si es white, le toca a black
  def update_movements(self, state):
    #print(state.board.GetMoves('W' if state.to_move == 'B' else 'B'))
    return state.board.GetMoves('W' if state.to_move == 'B' else 'B')

  #No da los movimientos necesariamente posibles, sino da los disponibles por pieza.
  #Esto porque la funcion GenericMove se encarga de analizarlos
  def actions(self, state):
    "Retorna moves porque estate es de la tupla GameState"
    return state.moves

  def result(self, state, move):
    if move not in state.moves: return state #No hay cambios
    board = state.board.Clon()
    board.GenericMove(move)
    moves = self.update_movements(state)
    return GameState(to_move=('W' if state.to_move == 'B' else 'B'),
                      utility=self.compute_utility(board, move, state.to_move),
                      board=board, moves=moves)

  def utility(self, state, player):
    """Retorna la utilidad del player en estado terminal state; 1 si ganó, -1 si perdió, 0 empate."""
    if player == 'W': return state.board.puntaje_blancas - state.board.puntaje_negras
    if player == 'B': return state.board.puntaje_negras - state.board.puntaje_blancas

  def terminal_test(self, state):
    """Un estado es terminal si hay un ganador o no hay mas movidas posibles."""
    return len(state.moves) == 0

  def display(self, state):
    board = state.board
    board.ImprimirTablero()

  def compute_utility(self, board, move, player):
    if player == 'W': return board.puntaje_blancas - board.puntaje_negras
    if player == 'B': return board.puntaje_negras - board.puntaje_blancas


## Algoritmo Monte Carlo

Implementación del Nodo

In [ ]:
class MCT_Node:
  """Nodo del árbol de búsqueda Monte Carlo. Hace un seguimiento de los estados hijos (`children` states)."""
  def __init__(self, parent=None, state=None, U=0, N=0):
    self.__dict__.update(parent=parent, state=state, U=U, N=N)
    self.children = {} #No hay hijos al inicio
    self.actions = None

Implementación de la función UCB1 para la selección. (Límite de confianza superior)

In [ ]:
import numpy as np
#Función donde n es el nodo y C es una constante
#n.parent.N es el total de simulaciones, y
def ucb(n, C=1.4):
  """Función UCB para la fase de selección."""
  if n.N == 0: return np.inf
  else: return (n.U / n.N) + C * np.sqrt(np.log(n.parent.N) / n.N)

Algoritmo MCTS

In [ ]:
import random

#N es la cantidad de simulaciones
def monte_carlo_tree_search(state, game, N=20, m=20):
  def select(n):
    """Selecciona un nodo del árbol."""
    if n.children: return select(max(n.children.keys(), key=ucb)) #Retorna el mejor nodo
    else: return n #Si no hay hijos se retorna a sí mismo

  def expand(n):
    """Expande la rama agregando todos sus estados hijo"""
    if not n.children and not game.terminal_test(n.state): #Si el juego no se acabó
      n.children = {MCT_Node(state=game.result(n.state, action), parent=n): action
                    for action in game.actions(n.state)} #Crea un hijo por accion posible
    return select(n)

  def simulate(game, state):
    """Simula la utilidad del estado actual al tomar aleatoriamente un paso."""
    player = game.to_move(state)
    i = 0 #Contador de iteraciones (para medir profundidad)
    while not game.terminal_test(state) and i < m: #Mientras que no se acabe
      action = random.choice(list(game.actions(state))) #Elige una accion aleatoria
      state = game.result(state, action)
      if game.terminal_test(state) and state.to_move == player: return -1000 #Hizo jaque mate
      elif game.terminal_test(state): return 1000 #Le hicieron jaque mate
      i += 1
    #print(player, game.utility(state, player), state.board.puntaje_blancas, state.board.puntaje_negras, action)
    v = game.utility(state, player)
    return -v #Retorna la puntuación en negativo, porque la recibirá min

  def backprop(n, utility):
    """Pasa la utilidad a todos los nodos padre (es decir, hacia atrás)."""
    if utility > 0: n.U += utility
    n.N += 1
    if n.parent: backprop(n.parent, -utility)

  root = MCT_Node(state=state)

  for _ in range(N): #Se expande N veces
    leaf = select(root)
    child = expand(leaf)
    result = simulate(game, child.state)
    backprop(child, result)

  max_state = max(root.children, key=lambda p: p.U)
  return root.children.get(max_state)

## Players

In [ ]:
def mcts_player(game, state):
  #print(state.moves)
  return monte_carlo_tree_search(state, game)

def random_player(game, state):
  a = random.choice(game.actions(state))
  #print(a)
  return a

def human_player(game, state):
  print("Estado actual:")
  game.display(state)
  print("Movidas disponibles: {}".format(game.actions(state)))
  print("")
  move_string = input('Humano, ¿Cuál es tu movida?')
  try: move = eval(move_string)
  except NameError: move = move_string
  return move

## Game Simulator

In [ ]:
from time import time
tiempo_inicial = time()

qchess = QuantumChess()
print(qchess.play_save_game(mcts_player, random_player, verbose=True))

tiempo_final = time()
tiempo_ejecucion = tiempo_final - tiempo_inicial
print('--- El tiempo de ejecucion fue:',tiempo_ejecucion,"segundos. ---")

Jugada 1 : Turno del jugador mcts_player ( W )
1 R K B Q E B K R 
2 P P P P P P P P 
3 . . . . . . . . 
4 . . . . . . . . 
5 . P . . . . . . 
6 . . . . . . . . 
7 P . P P P P P P 
8 R K B Q E B K R 
  A B C D E F G H
*************************************************
Jugada 2 : Turno del jugador random_player ( B )
1 R K B Q E B . R 
2 P P P P P P P P 
3 . . . . . K . K 
4 . . . . . . . . 
5 . P . . . . . . 
6 . . . . . . . . 
7 P . P P P P P P 
8 R K B Q E B K R 
  A B C D E F G H
*************************************************


KeyboardInterrupt: 